In [2]:
from random import seed
from random import randrange
from csv import reader
from math import exp

In [22]:
#load data into csv_reader
def load_data_from_csv_file(file):
  dataset=list()
  with open(file,'r') as file:
    csv_reader=reader(file)
    for row in csv_reader :
      if not row:
        continue
      dataset.append(row)
  return dataset

dataset=load_data_from_csv_file("diabetes.csv")
dataset=dataset[1:] #remove the feature name
dataset[0]

['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1']

In [23]:
#convert string into float for the dataset
def change_data_type(dataset,column):
    for row in dataset:
        row[column]=float(row[column].strip())


for i in range(len(dataset[0])):
  change_data_type(dataset,i)

#dataset[0]

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0]

In [27]:
# return the max and min for each feature
def find_minmax(dataset):
  minmax_list=list()
  for i in range(len(dataset[0])):
      values_in_each_column=[row[i] for row in dataset]
      min_value=min(values_in_each_column)
      max_value=max(values_in_each_column)
      minmax_list.append([min_value,max_value])
  return minmax_list

minmax_list=find_minmax(dataset)
#minmax_list

[[0.0, 17.0],
 [0.0, 199.0],
 [0.0, 122.0],
 [0.0, 99.0],
 [0.0, 846.0],
 [0.0, 67.1],
 [0.078, 2.42],
 [21.0, 81.0],
 [0.0, 1.0]]

In [28]:
#rescale data into range 0-1
def rescale_data(dataset,minmax_list):
    for row in dataset:
        for i in range(len(dataset[0])):
            row[i]=(row[i]-minmax_list[i][0])/(minmax_list[i][1]-minmax_list[i][0])

rescale_data(dataset,minmax_list)
#dataset[0]


[0.35294117647058826,
 0.7437185929648241,
 0.5901639344262295,
 0.35353535353535354,
 0.0,
 0.5007451564828614,
 0.23441502988898377,
 0.48333333333333334,
 1.0]

In [42]:
# n fold training dataset and testing dataset
def cross_validation(dataset,n_fold):
    splited_dataset=list() 

    copy_dataset=list(dataset)
    every_fold_length=int(len(dataset))/n_fold

    for i in range(n_fold):
        current_fold_dataset=list()
        while len(current_fold_dataset)<every_fold_length:
              if len(copy_dataset)==0:
                  break
              index=randrange(len(copy_dataset))
              data_sample=copy_dataset.pop(index)
              current_fold_dataset.append(data_sample)
        splited_dataset.append(current_fold_dataset)

    return splited_dataset

splited_dataset=cross_validation(dataset, 5)

#splited_dataset[0]
len(dataset)
len(splited_dataset[0])


154

In [46]:
# accuracy of the model
#pass in two list
#return a number

def accuracy(true_data, prediction):
  count=0
  for i in range(len(true_data)):
      if true_data[i]==prediction[i]:
        count=count+1
      
  return count/float(len(true_data)) *100



In [57]:
for fold in splited_dataset:
  training_dataset=list(splited_dataset)
  training_dataset.remove(fold)
  training_dataset=sum(training_dataset,[])
  print(len(training_dataset))

614
614
614
614
616


In [68]:
#evaluate the model
def evaluate_model(dataset,model,num_fold,*args):
    splitted_dataset=cross_validation(dataset,num_fold)
    score=list()
    for fold in splitted_dataset:
      training_dataset=list(splitted_dataset)
      training_dataset.remove(fold)
      training_dataset=sum(training_dataset,[]) #usning sum, combine the training_dataset
      testing_dataset=list()

      for row in fold:
          row_copy=list(row)
          testing_dataset.append(row_copy)
          row_copy[-1]=None
      
      prediction=model(training_dataset,testing_dataset,*args)
      actual=[row[-1] for row in fold]
      acc=accuracy(actual, prediction)
      score.append(acc)
    
    return score



In [69]:
#logistic regression model

def prediction(row, coefficients):
  yhat=coefficients[0]
  for i in range(len(row)-1):
    yhat=yhat+coefficients[i+1]*row[i]

  return 1/(1.0+exp(-yhat))

def using_sgd_update_coefficients(training_data,learning_rate,num_epoch):
  coefficients=[0.0 for i in range(len(training_data[0]))]
  for epoch in range(num_epoch):
    for row in training_data:
      yhat=prediction(row,coefficients)
      error=row[-1]-yhat
      coefficients[0]=coefficients[0] + learning_rate*error*yhat*(1.0-yhat)
      for i in range(len(row)-1):
        coefficients[i+1]=coefficients[i+1]+learning_rate*error*yhat*(1.0-yhat)*row[i]
  return coefficients

def logistic_regression(training_data,testing_data,learning_rate,num_epoch):
  predictions=list()
  coef=using_sgd_update_coefficients(training_data,learning_rate,num_epoch)
  for row in testing_data:
    yhat=prediction(row,coef)
    yhat=round(yhat)
    predictions.append(yhat)

  return predictions

In [70]:
seed(1)
dataset=load_data_from_csv_file("diabetes.csv")
dataset=dataset[1:] #remove the feature name
for i in range(len(dataset[0])):
  change_data_type(dataset,i)

minmax_list=find_minmax(dataset)
rescale_data(dataset,minmax_list)

num_fold=10
learning_rate=0.1
num_epoch=100

score=evaluate_model(dataset,logistic_regression,num_fold,learning_rate,num_epoch)


In [72]:
print("average accuracy is :", sum(score)/len(score))

average accuracy is : 77.21385281385282
